<a href="https://colab.research.google.com/github/RodolfoFerro/satelitesyneuronas/blob/main/notebooks/Redes_neuronales_artificiales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neuronales artificiales

### **Exploración de TensorFlow Playground**

Comenzaremos explorando el website: https://playground.tensorflow.org/

In [ ]:
from IPython.display import IFrame

IFrame(src="https://playground.tensorflow.org/", width="100%", height="600px")

### **Introducción a TensorFlow**

[TensorFlow](https://www.tensorflow.org/) es un framework open-source para Machine Learning desarrollada por Google. Utilizada para construir y entrenar redes neuronales artificiales gracias a su API de alto nivel, [Keras](https://keras.io/).

Comenzaremos creando una muestras, buscando resolver el [problema de separabilidad lineal para la compuerta lógica XOR](https://powerhousedm.blogspot.com/2007/10/el-problema-xor.html).

In [ ]:
import numpy as np

np.random.seed(123)
x = np.array([]) # TODO: Add samples for XOR
y = np.array([]) # TODO: Add samples for XOR

Utilizaremos Keras, la API de alto nivel subyaciente de TensorFlow, siendo la forma más común de utilizarlo.

Keras está compuesto de módulos, cada uno con diferentes objetos para diferentes necesidades:
- `models`
- `layers`
- `losses`
- `optimizers`

Estos módulos nos permiten construir, entrenar y evaluar redes neuronales de forma sencilla y modular. Por ejemplo, con `models` podemos definir arquitecturas secuenciales o funcionales; con `layers` agregamos capas como densas o convolucionales; `losses` define funciones de error como `mean_squared_error`, y `optimizers` permite elegir algoritmos de entrenamiento como Adam o SGD.

A continuación, exploraremos cómo usarlos en conjunto para construir un modelo básico en TensorFlow.

In [ ]:
import tensorflow as tf


model = tf.keras.models.Sequential([
    None, # Add Input of shape (2, )
    None, # Add Dense of size 2, activation=???
    None # Add Dense of size 1, activation=sigmoid
])

El modelo más sencillo para crear con Keras es `Sequential`. Este tipo de modelo permite apilar capas de forma lineal, donde la salida de una capa es la entrada de la siguiente. En el ejemplo, se define un modelo con una capa de entrada de dos dimensiones, seguida de una capa oculta densa con activación `tanh`, y una capa de salida con activación `sigmoid`, adecuada para tareas de clasificación binaria. Además, se fija una semilla aleatoria para asegurar la reproducibilidad del modelo.

Más referencias:
- Capas de redes en [Keras](https://keras.io/api/layers/) y [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/layers).
- Funciones de activación de [Keras](https://keras.io/api/layers/activations/) y [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/activations).

Este modelo, aún no entrenado, por sí solo ya puede generar una salida, dada una entrada de datos con la estructura correcta.

In [ ]:
one_one = np.array([[1, 1]])
model.predict(one_one)

### **Optimizador y función de pérdida**

Keras y TensorFlow cuentan con diferentes funciones de pérdida ya implementadas, por ejemplo la función `log-loss` que vimos previamente o la función de MSE (el error cuadrático medio).

Más referencias:
- Funciones de pérdida en [Keras](https://keras.io/api/losses/) y [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/losses).
- Optimizadores en Keras [Keras](https://keras.io/api/optimizers/) y [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).

In [ ]:
log_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

$$\text{log loss}=L_{\log}(y, p) = -(y \log(p) + (1 - y) \log(1 - p)) $$

In [ ]:
log_loss([1], [0])

In [ ]:
loss = tf.keras.losses.MeanSquaredError()

$$ \mathrm{MSE}=\frac{1}{N}\cdot\sum_{i=1}^N \left(y_i- \hat{y}_i \right )^2 $$

In [ ]:
loss([0.5], [0.5])

Podemos crear asimismo un optimizador, es decir, el método con el que buscaremos minimizar el error.

De nuestra sesión anterior, hablamos sobre el descenso de gradiente, por lo que utilizar el SDG será una buena opción para nuetsro problema.

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.6)

model.compile(optimizer=optimizer, loss='mse', metrics=[loss])

Podemos utilizar el método `.summary()` para ver una tabla con un resumen del modelo:

In [ ]:
model.summary()

Para entrenar el modelo, utilizamos la función `fit`, siendo consistentes con los modelos creados en `sklearn`.

In [ ]:
history = model.fit(x, y, epochs=1_000)

Una vez entrenado el modelo, podemos explorar la historia de entrenamiento, observando la disminución del error a lo largo de las épocas de entrenamiento.

In [ ]:
import plotly.express as px


losses = history.history["loss"]
eje_x = np.arange(len(losses))

fig = px.line(
    x=eje_x,
    y=losses,
    title="Historia de entrenamiento",
    labels=dict(x="Épocas", y="Error")
)
fig.show()

Ahora, evaluaremos el modelo en una rejilla para verificar la función aprendida.

In [ ]:
# Construcción de una rejilla
x = np.linspace(-0.2, 1.2, 201)
y = np.linspace(-0.2, 1.2, 201)
xy = np.meshgrid(x, y)
zz = np.array(list(zip(*(x.flat for x in xy))))

# Predicción en la rejilla de valores
surface = model.predict(zz)
surface = surface.flatten()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter3d(
    x=zz[:, 0],
    y=zz[:, 1],
    z=surface,
    mode="markers",
    marker=dict(
        size=1,
        color=surface,
        colorscale="Viridis",
        opacity=0.8
    )
)])

# Tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

<center>
    *********
</center>

### El dataset a utilizar: Fashion MNIST

El dataset está compuesto por imágenes de 28x28 pixeles, que contienen un conjunto de prendas en 10 categorías.


Los datos de Fashion MNIST están disponibles directamente en la API de conjuntos de datos de `tf.keras`. Los cargas así:

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

Llamar a `load_data` en este objeto nos dará dos conjuntos con los valores de entrenamiento y prueba para los gráficos que contienen las prendas y sus etiquetas.

In [ ]:
(training_images, training_labels), (test_images, test_labels) = fashion_mnist.load_data()

### El dataset a utilizar: Trees in Satellite Imagery

El dataset está compuesto por imágenes de 64x64 pixeles, que contienen un conjunto de 2 categorías.

El dataset fue descargado de: https://www.kaggle.com/datasets/mcagriaksoy/trees-in-satellite-imagery


In [ ]:
!wget https://github.com/RodolfoFerro/satelitesyneuronas/raw/refs/heads/main/assets/data/trees.zip

In [ ]:
!unzip trees
!mv Trees\ in\ Satellite\ Imagery trees_dataset

In [ ]:
!ls

In [ ]:
import os
import cv2
import numpy as np

dataset_path = 'trees_dataset'
classes = ['Trees', 'NoTrees']
img_size = 64
images = []
labels = []

for label, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

print(f"Loaded {len(images)} images with shape {images.shape}")
print(f"Loaded {len(labels)} labels with shape {labels.shape}")

In [ ]:
from sklearn.model_selection import train_test_split

training_images, testing_images, training_labels, testing_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training images shape: {training_images.shape}")
print(f"Testing images shape: {testing_images.shape}")
print(f"Training labels shape: {training_labels.shape}")
print(f"Testing labels shape: {testing_labels.shape}")

¿Cómo se ven estos valores?

Imprimamos una imagen de entrenamiento y una etiqueta de entrenamiento para ver.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=200)


# Set index of image to be seen
img_index = 0

# Plot image
plt.imshow(training_images[img_index], cmap='gray')
plt.axis(False)

print('Label:', training_labels[img_index])
print('Matrix:', training_images[img_index])

### Preparación de los datos

Notarás que todos los valores están entre 0 y 255. Si estamos entrenando una red neuronal, por varias razones es más fácil si transformamos los valores para tratar todos con valores entre 0 y 1. Este proceso se llama **normalización**.

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=200)


# Set index of image to be seen
img_index = 0

# Plot image
plt.imshow(training_images[img_index], cmap='gray')
plt.axis(False)

print('Label:', training_labels[img_index])
print('Matrix:', training_images[img_index])

In [ ]:
training_images[0].shape

### Creación del modelo



In [ ]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    # TODO. Dense -> 256, ReLU
    # TODO. Dense -> 2, Sigmoid
])

### Entrenamiento del modelo

Para entrenar el modelo, simplemente utilizamos el método `.fit()` del modelo.

In [ ]:
mlp_model.compile(
    optimizer=tf.optimizers.SGD(),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = mlp_model.fit(training_images, training_labels, epochs=3)


> **Pregunta clave:** ¿Qué sucede con la historia de entrenamiento?

In [ ]:
import plotly.express as px


seen = "accuracy" # or 'loss'

hist_values = history.history[seen]
eje_x = np.arange(len(hist_values))

fig = px.line(
    x=eje_x,
    y=hist_values,
    title="Historia de entrenamiento",
    labels=dict(x="Épocas", y=seen.capitalize())
)
fig.show()

### Evaluación del modelo

In [ ]:
mlp_model.evaluate(test_images, test_labels)

### Predicción


In [ ]:
import random

test_index = random.randint(0, 2080 - 1)

plt.imshow(test_images[test_index])
plt.axis(False)

print("Etiqueta:", test_labels[test_index])
input_image = np.reshape(test_images[test_index], (1, 4096))
prediction = mlp_model.predict(np.expand_dims(input_image, axis=-1))
print("Predicción:", np.argmax(prediction))

In [ ]:
prediction

> **Para resolver la tarea, el reto es:** Mejor accuracy obtenido en la clase.

**Puedes explorar:**
- El número de capas.
- Las épocas de entrenamiento.
- Las funciones de activación.
- Investigar otras capas.

---
> Contenido creado por **Rodolfo Ferro**. Contacto: [ferro@cimat.mx](ferro@cimat.mx) <br>
[**Clubes de Ciencia México**](https://clubesdeciencia.mx/), 2025.